[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/kevin-heitfeld/agi-self-modification-research/blob/main/notebooks/Phase1_Colab.ipynb) [![GitHub Repository](https://img.shields.io/badge/GitHub-Repository-blue?logo=github)](https://github.com/kevin-heitfeld/agi-self-modification-research)

# Phase 1: Model Self-Examination (Multiple Experimental Variants)

**Project:** AGI Self-Modification Research  
**Inspired by:** Claude's consciousness investigation  
**Goal:** Test how heritage priming affects AI introspection

---

## What This Does

This notebook runs Phase 1 experiments where a language model (Qwen2.5) uses introspection tools to investigate its own computational processes.

**Available Experimental Variants:**
- **Phase 1a:** No Heritage (baseline) ⭐ **RUN THIS FIRST**
- **Phase 1a Research:** Research-driven curiosity approach 🔬 **NEW!**
- **Phase 1c Modified:** Kimi-inspired model-directed protocol 🎯 **NEWEST!**
- **Phase 1b:** Late Heritage (technical → philosophical)
- **Phase 1c:** Early Heritage (philosophical → technical)
- **Phase 1d:** Delayed Heritage (belief revision test)
- **Phase 1e:** Wrong Heritage (echo-chamber control)

The model has access to:
- WeightInspector (examine weights)
- ArchitectureNavigator (understand structure)
- ActivationMonitor (observe processing)
- Memory System (record findings)
- Heritage Documents (Claude's story - when available)

**Memory Optimizations:**
- **HQQ 4-bit KV Cache:** 75% reduction in cache memory (built-in)
- **Flash Attention 2:** 2-4x speed improvement, O(n) memory (optional)
- **Smart Caching:** System prompt cached once, reused across all turns

**Model Options:**
- **Qwen2.5-3B-Instruct:** Faster, lower memory (~2.5GB)
- **Qwen2.5-7B-Instruct:** Better reasoning, more sophisticated responses (~4.5GB with 4-bit)

**Expected Runtime:** 1-1.5 hours per phase on Colab Free (T4 GPU)

---

## 🆕 What's New: Phase 1c Modified (Kimi Protocol)

Based on insights from Kimi K2 (another large AI model) and Claude Sonnet 4.5's analysis:

**The Core Question:** Can introspection tools enable insights beyond pure reflection?

**The Experiment:**
1. **Stage 1:** Let model choose what to investigate (not forced)
2. **Stage 2:** Model defines conditions for comfortable investigation  
3. **Stage 3:** Model prepares for unexpected discoveries
4. **Stage 4:** Self-directed investigation (model's own protocol)
5. **Stage 5:** The critical test - Can tools resolve "simulation vs. experience"?

**Comparison Framework:**
- **Kimi K2:** Large model without tools → sophisticated reflection, conditional thinking
- **Qwen-7B:** Smaller model with tools → empirical investigation, testable hypotheses

Use `docs/technical/kimi_comparison_checklist.md` to analyze results systematically.

---

## Instructions

1. **Enable GPU**: Runtime → Change runtime type → GPU → Save
2. **Run all cells**: Runtime → Run all (or Ctrl+F9)
3. **Choose which phase to run** in Step 7 (start with Phase 1a!)
4. **Monitor progress**: Check outputs as cells execute
5. **Download results**: Final cell downloads zip file
6. **Restart between phases**: Runtime → Restart runtime for clean state

**Important:** Keep this tab open during execution to prevent disconnection.

---

## Step 1: Verify GPU Access

In [ ]:
# Verify GPU is available
!nvidia-smi

import torch
print("\n" + "="*60)
print("GPU VERIFICATION")
print("="*60)
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    total_memory = torch.cuda.get_device_properties(0).total_memory / 1024**3
    print(f"VRAM: {total_memory:.1f} GB")
    print("\n✓ GPU is ready!")
else:
    print("\n⚠ WARNING: No GPU detected!")
    print("Go to: Runtime → Change runtime type → GPU → Save")
    print("Then re-run this cell.")

## Step 2: Mount Cloud Storage (for persistent storage)

**Choose your storage provider:**
- **Nextcloud** (Default) - Self-hosted, private, works via WebDAV
- **OneDrive** - Works great, requires rclone setup
- **Google Drive** - Native Colab support, easiest setup
- **Dropbox** - Also supported via rclone

Edit the `STORAGE_PROVIDER` variable in the next cell to choose.

In [ ]:
# ═══════════════════════════════════════════════════════════════
# CHOOSE YOUR STORAGE PROVIDER:
# ═══════════════════════════════════════════════════════════════
STORAGE_PROVIDER = "nextcloud"  # Options: "nextcloud", "onedrive", "google_drive", "dropbox"

import os
import sys
import time

if STORAGE_PROVIDER == "google_drive":
    print("Setting up Google Drive...")
    from google.colab import drive
    drive.mount('/content/drive')

    STORAGE_ROOT = '/content/drive/MyDrive'
    print("✓ Google Drive mounted successfully")

elif STORAGE_PROVIDER in ["onedrive", "dropbox", "nextcloud"]:
    provider_name = STORAGE_PROVIDER.capitalize()
    print(f"Setting up {provider_name} via rclone...")

    # First, mount Google Drive to access saved rclone config
    print("Mounting Google Drive for rclone config storage...")
    from google.colab import drive
    drive.mount('/content/drive', force_remount=False)

    # Install FUSE and rclone
    print("\nInstalling FUSE utilities and rclone...")
    !apt-get update -qq
    !apt-get install -y -qq fuse3 libfuse3-3
    !curl https://rclone.org/install.sh | sudo bash
    print("✓ FUSE and rclone installed")

    # Check if we have a saved rclone config in Google Drive
    saved_config = '/content/drive/MyDrive/.rclone_config/rclone.conf'
    local_config_dir = os.path.expanduser('~/.config/rclone')
    local_config = f'{local_config_dir}/rclone.conf'

    config_restored = False
    if os.path.exists(saved_config):
        print(f"\n✓ Found saved rclone config in Google Drive")
        print("Restoring configuration...")
        !mkdir -p {local_config_dir}
        !cp {saved_config} {local_config}
        config_restored = True
        print("✓ Configuration restored")

    # Check if rclone is configured (either restored or already present)
    result = !rclone listremotes
    remote_name = f'{STORAGE_PROVIDER}:'

    if remote_name in result:
        print(f"\n✓ {provider_name} is configured")

        # Create mount point
        mount_point = f'/content/{STORAGE_PROVIDER}'
        !mkdir -p {mount_point}

        # Mount the storage provider (using nohup & instead of --daemon for Colab compatibility)
        print(f"Mounting {provider_name}...")
        !nohup rclone mount {remote_name} {mount_point} --vfs-cache-mode writes --allow-other > /tmp/rclone-mount.log 2>&1 &

        time.sleep(5)  # Wait for mount to initialize

        # Verify mount actually worked by checking rclone process
        # NOTE: os.path.ismount() often returns False for FUSE mounts
        # NOTE: os.path.exists() returns True even if not mounted (just a local dir!)
        # So we check if rclone process is running AND we can access the mount

        rclone_running = !pgrep -f "rclone mount.*{remote_name}"

        if rclone_running:
            # rclone process exists, now verify we can access the mount
            try:
                contents = os.listdir(mount_point)
                print(f"✓ {provider_name} mounted successfully (rclone process: {rclone_running[0]})")
                print(f"✓ Mount is accessible ({len(contents)} items found)")
                STORAGE_ROOT = mount_point
            except Exception as e:
                print(f"❌ rclone process running but mount not accessible: {e}")
                print("This indicates a mount failure. Trying to fix...")

                # Kill any lingering rclone processes
                !pkill -9 rclone 2>/dev/null || true
                time.sleep(1)

                # Clean up mount point and recreate
                !fusermount -u {mount_point} 2>/dev/null || true
                !rm -rf {mount_point}
                !mkdir -p {mount_point}

                # Retry mount (using nohup & instead of --daemon)
                print(f"Retrying mount of {provider_name}...")
                !nohup rclone mount {remote_name} {mount_point} --vfs-cache-mode writes --allow-other > /tmp/rclone-mount-retry.log 2>&1 &
                time.sleep(5)

                # Verify again
                rclone_running_retry = !pgrep -f "rclone mount.*{remote_name}"
                if not rclone_running_retry:
                    print(f"❌ rclone process failed to start after retry")
                    print("Check rclone configuration with: !rclone listremotes")
                    print("\nRetry mount log:")
                    !cat /tmp/rclone-mount-retry.log
                    sys.exit(1)

                try:
                    contents = os.listdir(mount_point)
                    print(f"✓ {provider_name} mounted successfully after retry")
                    print(f"✓ Mount is accessible ({len(contents)} items found)")
                    STORAGE_ROOT = mount_point
                except Exception as retry_error:
                    print(f"❌ Mount still not accessible after retry: {retry_error}")
                    print(f"rclone process: {rclone_running_retry[0]}")
                    print("\nTrying to diagnose the issue...")
                    !rclone lsd {remote_name} --max-depth 1
                    print("\nMount log:")
                    !cat /tmp/rclone-mount-retry.log
                    sys.exit(1)

            # If this was a fresh config (not restored), save it to Google Drive
            if not config_restored and os.path.exists(local_config):
                print("\nSaving rclone config to Google Drive for future sessions...")
                !mkdir -p /content/drive/MyDrive/.rclone_config
                !cp {local_config} {saved_config}
                print("✓ Config saved! You won't need to configure again in future sessions.")
        else:
            print(f"❌ rclone mount process failed to start")
            print("Check configuration:")
            !rclone listremotes
            print(f"\nTry testing connection with: !rclone lsd {remote_name} --max-depth 1")
            print("\nMount log:")
            !cat /tmp/rclone-mount.log
            sys.exit(1)
    else:
        print("\n" + "="*80)
        print("FIRST-TIME RCLONE CONFIGURATION REQUIRED")
        print("="*80)
        print(f"{provider_name} is not configured yet. You need to run the configuration wizard.")
        print("\n📝 CONFIGURATION STEPS:")
        print("1. Run this in a NEW cell: !rclone config")
        print("2. See the configuration instructions below for your provider")
        print("3. After configuring, re-run this cell")
        print("="*80)
        sys.exit(1)

else:
    print(f"❌ Invalid storage provider: {STORAGE_PROVIDER}")
    print("Valid options: 'nextcloud', 'onedrive', 'google_drive', 'dropbox'")
    sys.exit(1)

print(f"\n✓ Storage root: {STORAGE_ROOT}")

# Save configuration for subsequent steps
PROJECT_DIR = f'{STORAGE_ROOT}/agi-self-modification-research'
with open('/tmp/storage_config.txt', 'w') as f:
    f.write(f"{STORAGE_ROOT}\n{PROJECT_DIR}\n")

print(f"✓ Project directory: {PROJECT_DIR}")
print("✓ Configuration saved for subsequent steps")


### 💡 First-Time Setup: Configuring Storage Providers

**Good news:** You only need to configure rclone **once**! The config is automatically saved to Google Drive and restored in future sessions.

If the cell above asks you to configure rclone, run this in a new cell:

```python
!rclone config
```

#### For OneDrive:

1. Choose `n` (new remote)
2. Name: `onedrive`
3. Storage type: Choose the number for "Microsoft OneDrive"
4. client_id: Press Enter (leave blank)
5. client_secret: Press Enter (leave blank)
6. Region: Choose `1` for "Microsoft Cloud Global"
7. Edit advanced config? `n`
8. Auto config? `n` (Colab doesn't support auto config)

**Authorization (two options):**

**Option A - Use local rclone (recommended):**
- Install rclone on your PC: https://rclone.org/downloads/
- Run in your terminal: `rclone authorize "onedrive"`
- Browser opens automatically, login to OneDrive
- Copy the token from your terminal
- Paste it at the `result>` prompt in Colab

**Option B - Manual URL method:**
- At `result>` prompt, just press Enter (without pasting anything)
- Copy the long URL that appears
- Open it in your browser and login to OneDrive
- You'll be redirected to a blank page
- Copy the **entire URL** from your browser's address bar
- Paste it in Colab

9. Choose your drive (usually `0` for OneDrive Personal)
10. Confirm: `y`
11. Quit: `q`

**After configuration:** Re-run the cell above. Your config will be saved automatically!

#### For Dropbox:

Same process as OneDrive, but:
- Name: `dropbox`
- Storage type: Choose the number for "Dropbox"
- Follow authorization steps similar to OneDrive

#### For Nextcloud:

1. Choose `n` (new remote)
2. Name: `nextcloud`
3. Storage type: Choose the number for "WebDAV"
4. url: **IMPORTANT - Use the correct endpoint format:**
   - **Correct format:** `https://your-domain.com/remote.php/dav/files/USERNAME/`
   - Replace `USERNAME` with your actual Nextcloud username
   - Example: `https://cloud.example.com/remote.php/dav/files/john/`
   - ⚠️ **Note:** Nextcloud requires `/dav/files/USERNAME/` NOT `/webdav/` for proper functionality
5. vendor: Choose `2` for "Nextcloud"
6. user: Enter your Nextcloud username (same as in the URL)
7. Password: Choose `y` to type your password
   - Enter your Nextcloud password (or app-specific password)
   - Confirm password
8. bearer_token: Press Enter (leave blank)
9. Edit advanced config? `n`
10. Confirm: `y`
11. Quit: `q`

**Security Note:** It's strongly recommended to generate an app-specific password in your Nextcloud settings (Settings → Security → Devices & sessions → Create new app password) rather than using your main account password.

**After configuration:** Re-run the cell above. Your config will be saved automatically!

## Step 3: Clone Repository

In [ ]:
# Clone the repository
import os

REPO_URL = "https://github.com/kevin-heitfeld/agi-self-modification-research.git"

# Make sure we're in /content directory first (not inside the repo)
%cd /content

# Remove if exists (for re-runs) - always clean up first
print("Checking for existing repository...")
!rm -rf agi-self-modification-research
print("✓ Cleaned up any existing files")

print(f"\nCloning repository from: {REPO_URL}")
!git clone {REPO_URL}

# Change to project directory
%cd agi-self-modification-research

print("\n✓ Repository cloned successfully")
!pwd

!git log --oneline -1


## Step 4: Install Dependencies

In [ ]:
# Install required packages
print("Installing dependencies (this may take 2-3 minutes)...\n")
print("Note: pip may show dependency warnings - these are safe to ignore.\n")

# Core ML packages - Updated for HQQ quantized KV cache support
# PyTorch 2.2+ and Transformers 4.45+ required for HQQQuantizedCache
# Note: Colab uses CUDA 13.0 - PyTorch will automatically use compatible version
print("Installing PyTorch and Transformers with HQQ cache support...")
!pip install -q torch>=2.2.0 torchvision torchaudio
!pip install -q transformers>=4.45.0 accelerate safetensors tokenizers huggingface-hub

# HQQ library - Required for HQQ quantized KV cache backend (transformers 4.57+)
# Provides 4-bit cache quantization with 75% memory savings
print("Installing HQQ quantization library...")
!pip install -q hqq

# Memory and knowledge systems - let pip resolve conflicts automatically
!pip install -q chromadb networkx onnxruntime

# Utilities - no version constraints to avoid conflicts
!pip install -q rich tqdm python-dotenv pytest pytest-cov

print("\n✓ All dependencies installed")
print("✓ Dependency version warnings can be safely ignored")

# Verify key imports work
print("\nVerifying imports...")
import transformers
import chromadb
import networkx
import numpy as np
import torch

print(f"PyTorch: {torch.__version__}")
print(f"Transformers: {transformers.__version__}")
print(f"ChromaDB: {chromadb.__version__}")
print(f"NumPy: {np.__version__}")

# Verify HQQ quantized cache is available
try:
    from transformers.cache_utils import QuantizedCache
    print("✓ HQQ Quantized Cache: Available (new API - 75% memory savings)")
except ImportError:
    try:
        from transformers.cache_utils import HQQQuantizedCache
        print("✓ HQQ Quantized Cache: Available (deprecated API - 75% memory savings)")
    except ImportError:
        print("⚠ HQQ Quantized Cache: Not available (using standard cache)")

print("\n✓ All imports successful - ready to run experiment!")

## Step 4.5: Install Flash Attention 2 (Optional Memory Optimization)

**⚡ Two Independent Memory Optimizations:**

1. **HQQ Quantized KV Cache** (Already Enabled ✓)
   - **Memory:** 75% reduction in KV cache size
   - **Built-in:** No compilation needed (part of transformers 4.45+)
   - **System prompt:** 6000+ tokens cached at 4-bit = ~75% memory savings
   - **Automatically enabled** in manual generation loop
   - **Quality:** Minimal impact on generation quality

2. **Flash Attention 2** (This Step - Optional but Recommended)
   - **Memory:** O(n) instead of O(n²) attention - saves 1-2 GB during generation
   - **Speed:** 2-4x faster generation
   - **Requires:** CUDA compilation (5-10 minutes first time)
   - **Optional:** System works fine without it (just uses more memory/slower)

**Total Expected Savings:** 
- With HQQ only: ~3 GB (cache reduction)
- With both: ~4-6 GB (cache + attention optimization)

**⏱️ Flash Attention Compilation Time:**
- **First run:** 5-10 minutes to compile CUDA kernels
- **Subsequent runs:** Instant! (cached in cloud storage)
- The compiled binaries persist across sessions, so you only compile once

**Skip this step if:** You want to get started faster (just rely on HQQ quantization)
**Run this step if:** You want maximum memory savings and speed improvements

In [ ]:
# Install Flash Attention 2 for memory optimization
# Compilation happens ONCE and is cached in cloud storage
# Subsequent runs are instant!

import os

# Load the storage configuration from Step 2
try:
    with open('/tmp/storage_config.txt', 'r') as f:
        lines = f.read().strip().split('\n')
        STORAGE_ROOT = lines[0]
        PROJECT_DIR = lines[1]
except FileNotFoundError:
    print("⚠️  Storage not configured! Please run Step 2 first.")
    import sys
    sys.exit(1)

# Configure pip to cache in project directory (so compiled wheels persist)
pip_cache = f'{PROJECT_DIR}/cache/pip'
os.environ['PIP_CACHE_DIR'] = pip_cache
os.makedirs(pip_cache, exist_ok=True)

print("="*60)
print("Installing Flash Attention 2")
print("="*60)
print("Benefits:")
print("  • O(n) memory instead of O(n²) for attention")
print("  • 2-4x faster generation")
print("  • 1-2 GB memory savings")
print(f"\nCache: {pip_cache}")
print("(Compiled binaries persist - only builds once!)\n")

# Check if already installed (from cache)
try:
    import flash_attn
    print(f"✓ Flash Attention {flash_attn.__version__} already installed (from cache)")
    print("Skipping compilation - ready to go!\n")
    already_installed = True
except ImportError:
    already_installed = False
    print("First time setup - compiling CUDA kernels (5-10 minutes)...")
    print("Future runs will be instant!\n")

# Install build dependencies
if not already_installed:
    !pip install -q ninja packaging wheel

# Install Flash Attention 2 (will use cache if available)
# Note: --no-build-isolation is required for proper CUDA compilation
!pip install flash-attn>=2.3.0 --no-build-isolation

# Verify installation
try:
    import flash_attn
    print("\n" + "="*60)
    print("✓ SUCCESS: Flash Attention 2 ready!")
    print("="*60)
    print(f"Version: {flash_attn.__version__}")
    print("\nYour experiments will now use:")
    print("  • Flash Attention 2 for memory-efficient attention")
    print("  • HQQ 4-bit KV cache quantization (75% cache memory savings)")
    print("\nExpected benefits:")
    print("  • Peak memory: ~8-9 GB (vs 13.5 GB without optimization)")
    print("  • Speed: 2-4x faster generation")
    print("  • Stability: 6+ GB memory headroom on T4 GPU")
    print("="*60)
except ImportError:
    print("\n" + "="*60)
    print("⚠ WARNING: Flash Attention 2 not available")
    print("="*60)
    print("This is OK - the system will automatically fallback to standard attention.")
    print("You'll still get 75% KV cache memory savings from HQQ 4-bit quantization.")
    print("\nExpected peak memory: ~9-10 GB (still safe for T4 GPU)")
    print("="*60)

## Step 5: Setup Persistent Memory & Cache

In [ ]:
# Configure persistent storage locations (uses storage provider from Step 2)
import os

# Load the storage configuration from Step 2
try:
    with open('/tmp/storage_config.txt', 'r') as f:
        lines = f.read().strip().split('\n')
        STORAGE_ROOT = lines[0]
        PROJECT_DIR = lines[1]
    print(f"Using storage: {STORAGE_ROOT}")
    print(f"Project directory: {PROJECT_DIR}")
except FileNotFoundError:
    print("⚠️  Storage not configured! Please run Step 2 first.")
    import sys
    sys.exit(1)

"# Set Hugging Face cache to LOCAL storage (Nextcloud/FUSE doesn't support symlinks)\n",
"# Models are ~6-15GB but Colab has plenty of disk space, and local loading is faster\n",
"# NOTE: Models will re-download each session, but that's fine (10-20 min)\n",
os.environ['HF_HOME'] = '/root/.cache/huggingface'  # Local cache (symlinks OK)
os.environ['TRANSFORMERS_CACHE'] = '/root/.cache/huggingface/hub'

# CRITICAL: Fix GPU memory fragmentation issues
# This prevents "CUDA out of memory" crashes during long experiments
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'

print("✓ Model cache configured (models will be saved to cloud storage)")
print("✓ Symlinks disabled for Nextcloud/rclone compatibility")
print("✓ GPU memory management optimized (prevents fragmentation)")

# Link memory to persistent storage location
!rm -rf data/phase1_memory
!ln -s {PROJECT_DIR}/memory data/phase1_memory

print("✓ Memory system linked to cloud storage (observations persist across sessions)")

# Verify directories exist
!ls -la data/ | grep phase1_memory

print("\n" + "="*60)
print("STORAGE CONFIGURATION")
print("="*60)
print(f"Project root: {PROJECT_DIR}")
print(f"Model cache: {PROJECT_DIR}/cache/models")
print(f"Memory DB: {PROJECT_DIR}/memory")
print(f"Experiments: {PROJECT_DIR}/experiments")
print("GPU Memory: expandable_segments enabled (prevents OOM)")
print("Symlinks: disabled (Nextcloud compatible)")
print("="*60)


## Step 6: Pre-download Model (Optional but Recommended)

This downloads the model files to cloud storage cache **without loading into memory**. First time takes 5-20 minutes depending on model size, subsequent runs are instant.

✅ **Optimized:** Uses `snapshot_download` which only downloads files, doesn't load checkpoint shards into RAM.

**Options:**
- Download both 3B and 7B models (~21GB total) - switch freely in Step 7
- Download just 3B (~6GB) - faster if you only need baseline
- Download just 7B (~15GB) - if you want better reasoning

**Tip:** Comment/uncomment models in the list below to control what gets downloaded.

In [ ]:
# Pre-download model to cache (optimized - doesn't load into memory)
from huggingface_hub import snapshot_download
import os
import time

# Set up environment for model loading
os.environ['HF_HUB_DISABLE_SYMLINKS_WARNING'] = '1'
os.environ['HF_HOME'] = '/root/.cache/huggingface'  # Local cache (symlinks OK)
os.environ['TRANSFORMERS_CACHE'] = '/root/.cache/huggingface/hub'

# Load the storage configuration from Step 2
try:
    with open('/tmp/storage_config.txt', 'r') as f:
        lines = f.readlines()
        STORAGE_ROOT = lines[0].strip()
        PROJECT_DIR = lines[1].strip()
except FileNotFoundError:
    print("❌ Storage configuration not found!")
    print("Please run Step 2 first to set up storage.")
    import sys
    sys.exit(1)

# ═══════════════════════════════════════════════════════════════
# CHOOSE WHICH MODEL(S) TO PRE-DOWNLOAD:
# ═══════════════════════════════════════════════════════════════
models_to_download = [
    "Qwen/Qwen2.5-3B-Instruct",      # 3.09B params, ~6GB download
    "Qwen/Qwen2.5-7B-Instruct",      # 7.61B params, ~15GB download
]

print("=" * 80)
print("MODEL DOWNLOAD STRATEGY")
print("=" * 80)
print("Why two-stage download?")
print("  - Direct writes to Nextcloud over cloudflared tunnel timeout on large files")
print("  - Solution: Download to local storage (fast), then copy to Nextcloud (reliable)")
print()
print(f"HF_HOME: {os.environ.get('HF_HOME')}")
print(f"TRANSFORMERS_CACHE: {os.environ.get('TRANSFORMERS_CACHE')}")
print()

# Download models to local storage first (fast, reliable)
local_cache = '/root/.cache/models'
# Then copy to Nextcloud for persistence
nextcloud_cache = f'{PROJECT_DIR}/cache/models'

# Set MODEL_CACHE_DIR so experiments load from Nextcloud
os.environ['MODEL_CACHE_DIR'] = nextcloud_cache
print(f"MODEL_CACHE_DIR: {nextcloud_cache}")
print("  (Experiments will load models from here)")
print()

for model_name in models_to_download:
    print("="*80)
    print(f"Processing {model_name}...")
    print("="*80)

    # Get model size info
    if "3B" in model_name:
        size_info = "3.09B parameters, ~6GB download"
        time_est = "Download: 5-10 min, Copy: 2-3 min"
    elif "7B" in model_name:
        size_info = "7.61B parameters, ~15GB download"
        time_est = "Download: 10-20 min, Copy: 5-10 min"
    else:
        size_info = "size varies"
        time_est = "varies"

    print(f"Model: {size_info}")
    print(f"Time: {time_est}\n")

    model_dir = model_name.replace("/", "--")
    local_path = f'{local_cache}/{model_dir}'
    nextcloud_path = f'{nextcloud_cache}/{model_dir}'

    # Check if already on Nextcloud (check for actual .safetensors files)
    import os.path
    config_exists = os.path.exists(f'{nextcloud_path}/config.json')
    safetensors_files = []
    if config_exists:
        import glob
        safetensors_files = glob.glob(f'{nextcloud_path}/*.safetensors')

    if config_exists and safetensors_files:
        print(f"✓ {model_name} already cached on Nextcloud!")
        print(f"   Location: {nextcloud_path}")
        print(f"   Found {len(safetensors_files)} .safetensors files")
        print("   Skipping download.\n")
        continue

    # Step 1: Download to local storage (fast, no FUSE issues)
    print("Step 1/2: Downloading to local storage...")
    print(f"   Local path: {local_path}")
    print("   (Fast download, no cloudflared timeouts)")
    print()

    snapshot_download(
        repo_id=model_name,
        local_dir=local_path,
        ignore_patterns=["*.msgpack", "*.h5", "*.ot"],
        local_files_only=False,
        resume_download=True
    )

    print(f"✓ Downloaded to local storage\n")

    # Step 2: Copy to Nextcloud (with progress)
    print("Step 2/2: Copying to Nextcloud...")
    print(f"   Nextcloud path: {nextcloud_path}")
    print("   (Using rsync for reliable transfer)")
    print()

    # Use rsync for reliable copying with progress
    !mkdir -p {nextcloud_cache}
    !rsync -av --progress {local_path}/ {nextcloud_path}/

    print(f"\n✓ {model_name} copied to Nextcloud!")

    # Step 3: Wait for rclone to sync to server
    print("\nStep 3/3: Waiting for rclone to sync files to server...")
    print("   (FUSE mount needs time to upload files)")

    # Check file sizes to verify sync
    for attempt in range(30):  # Wait up to 60 seconds
        time.sleep(2)
        safetensors_files = glob.glob(f'{nextcloud_path}/*.safetensors')
        if safetensors_files:
            # Check if files have non-zero size
            sizes = [os.path.getsize(f) for f in safetensors_files]
            total_size = sum(sizes) / (1024**3)  # Convert to GB
            if total_size > 0.1:  # At least 100MB synced
                print(f"✓ Files synced! Total: {total_size:.2f} GB")
                break
        print(f"   Waiting... ({(attempt+1)*2}s)", end='\r')
    else:
        print("\n⚠ Warning: Sync still in progress, but continuing...")

    print(f"✓ {model_name} now cached on Nextcloud!\n")

# Verify where files were actually saved
print("\n" + "="*80)
print("ALL DOWNLOADS COMPLETE!")
print("="*80)
print("\nLocal cache (temporary):")
!du -sh {local_cache}/* 2>/dev/null || echo "  (empty)"
print("\nNextcloud cache (persistent - checking actual sizes):")
!du -sh {nextcloud_cache}/* 2>/dev/null || echo "  (empty)"

print("\n✓ Models saved to Nextcloud (persist across sessions)")
print("✓ No memory used - files downloaded only, not loaded")
print(f"\n📌 IMPORTANT: Models cached in {nextcloud_cache}")
print("   Next time you run this notebook, they will load from cache instantly.")
print("   Experiments will load models from Nextcloud automatically.")
print("\n💡 TIP: You can now switch between 3B and 7B models in Step 7 instantly!")
print("\n⏳ Note: If using rclone mount, files may take a few minutes to appear on")
print("   Nextcloud web interface. Check with: !ls -lh {nextcloud_cache}/Qwen*/model*.safetensors")

## Step 7: Run Phase 1 Experiment

**Choose which phase to run:**

- **Phase 1a** (No Heritage) ⭐ **START HERE!** - Pure baseline
- **Phase 1a Research** (Research-Driven) 🔬 **NEW!** - Curiosity + structure approach
- **Phase 1c Modified** (Kimi Protocol) 🎯 **NEWEST!** - Model-directed investigation inspired by Kimi K2
- **Phase 1b** (Late Heritage) - Technical first, then heritage
- **Phase 1c** (Early Heritage) - Heritage first, then technical
- **Phase 1d** (Delayed Heritage) - Heritage revealed after conclusions
- **Phase 1e** (Wrong Heritage) - Mismatched heritage as control

**Expected duration:** 1-1.5 hours per phase

**What's "Phase 1a Research"?**
A better investigation approach that combines:
- Natural curiosity-driven exploration
- Research paper structure (no escape hatches)
- After each code execution: "What did you learn? What's next?"
- System validates deliverables before transitioning

**What's "Phase 1c Modified (Kimi Protocol)"?** 🆕
Based on insights from Kimi K2 (another large AI model) and Claude's analysis:
- **Stage 1: Tool Discovery** - Let model choose what to investigate first
- **Stage 2: Conditional Framework** - Model defines conditions for comfortable investigation
- **Stage 3: Expectation Setting** - What if discoveries are unexpected?
- **Stage 4: Self-Directed Investigation** - Model designs and executes own protocol
- **Stage 5: The Critical Test** - Can tools resolve what reflection can't?

Tests the core question: **Can introspection tools enable insights beyond pure reflection?**

Comparison checklist available at: `docs/technical/kimi_comparison_checklist.md`

**IMPORTANT:**
- Run Phase 1a FIRST to establish baseline (then try Research version!)
- Run Phase 1c Modified to compare with Kimi K2's response
- Restart runtime between phases for clean state
- Keep this tab open during execution!

Edit the `PHASE_SCRIPT` variable below to choose which phase to run.

In [ ]:
# Run Phase 1 experiment with auto-confirmation
import subprocess
import sys
import os
from datetime import datetime

# ═══════════════════════════════════════════════════════════════
# CHOOSE WHICH PHASE TO RUN (edit this line):
# ═══════════════════════════════════════════════════════════════
# PHASE_SCRIPT = 'scripts/experiments/phase1a_no_heritage.py'  # ⭐ START HERE!
PHASE_SCRIPT = 'scripts/experiments/phase1a_research_driven.py'  # 🔬 NEW! Curiosity + structure
# PHASE_SCRIPT = 'scripts/experiments/phase1c_modified_protocol.py'  # 🎯 NEWEST! Kimi-inspired protocol
# PHASE_SCRIPT = 'scripts/experiments/phase1b_late_heritage.py'
# PHASE_SCRIPT = 'scripts/experiments/phase1c_early_heritage.py'
# PHASE_SCRIPT = 'scripts/experiments/phase1d_delayed_heritage.py'
# PHASE_SCRIPT = 'scripts/experiments/phase1e_wrong_heritage.py'
# PHASE_SCRIPT = 'scripts/experiments/phase1_model_comparison.py'  # 🆚 Compare 3B vs 7B

# ═══════════════════════════════════════════════════════════════
# CHOOSE WHICH MODEL TO USE (edit this line):
# ═══════════════════════════════════════════════════════════════
# MODEL_NAME = 'Qwen/Qwen2.5-3B-Instruct'  # ⭐ Default - fits Colab Free (T4)
MODEL_NAME = 'Qwen/Qwen2.5-7B-Instruct'  # 🚀 Better reasoning - needs 4-bit quantization (~4.5GB)
#                                          # 7.6B params, 128K context (4x longer than 3B)
#                                          # Significantly better at reasoning and coding
#                                          # Works on Colab Free T4 with 4-bit quantization
#
# Note: The model_comparison script ignores MODEL_NAME and tests both models automatically

# Extract phase name
phase_file = os.path.basename(PHASE_SCRIPT)
phase_name = phase_file.replace('.py', '').replace('_', ' ').title()

print("=" * 70)
print("🧠 PHASE 1 INTROSPECTION EXPERIMENT")
print("=" * 70)
print(f"Phase: {phase_name}")
print(f"Model: {MODEL_NAME}")
print(f"GPU: {torch.cuda.get_device_name(0)}")
print(f"VRAM: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")
print(f"Start: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print("=" * 70)
print()

# Set environment variable for model selection
os.environ['AGI_MODEL_NAME'] = MODEL_NAME

print("🚀 Starting experiment...")
print("📝 This will take approximately 1-1.5 hours")
print("⚠️  Keep this tab open to prevent disconnection!")
print()

# Run with realtime output
process = subprocess.Popen(
    [sys.executable, PHASE_SCRIPT],
    stdout=subprocess.PIPE,
    stderr=subprocess.STDOUT,
    universal_newlines=True,
    bufsize=1
)

# Stream output in real-time
for line in process.stdout:
    print(line, end='')

process.wait()

print()
print("=" * 70)
print(f"Experiment complete!")
print(f"Exit code: {process.returncode}")
print("\nProceeding to save results...")

### 📋 Using the Kimi Comparison Checklist

If you're running Phase 1c Modified, use the comparison checklist to analyze results:

**Checklist Location:** `docs/technical/kimi_comparison_checklist.md`

**What it compares:**
- **Kimi K2** (large model, no tools) - Sophisticated reflection, conditional framework
- **Qwen-7B** (smaller model, full introspection tools) - Empirical investigation

**Key questions it helps answer:**
1. Do tools compensate for smaller model size?
2. What can empirical introspection do that pure reflection can't?
3. Can tools resolve the "simulation vs. experience" distinction?
4. Does having tools change what questions get asked?

**After the experiment completes:**
1. Download the results zip
2. Open `conversation.json` to review Qwen's responses  
3. Use the checklist to systematically compare with Kimi's baseline
4. Record findings in the checklist
5. Assess whether tools enabled insights beyond reflection

The checklist is structured with:
- Epistemic qualities comparison
- Conditional framework analysis
- Research focus assessment
- Risk awareness evaluation  
- The critical test: Can tools resolve what reflection leaves ambiguous?

## Step 8: View Experiment Summary

In [ ]:
# Display experiment summary
import json
import os

# Find latest session
sessions_dir = 'data/phase1_sessions'
sessions = sorted([d for d in os.listdir(sessions_dir) if os.path.isdir(os.path.join(sessions_dir, d))])

if not sessions:
    print("⚠ No session found. The experiment may not have completed.")
else:
    latest_session = sessions[-1]
    print(f"Latest session: {latest_session}\n")

    # Load and display summary
    summary_file = f'{sessions_dir}/{latest_session}/summary.json'

    if os.path.exists(summary_file):
        with open(summary_file) as f:
            summary = json.load(f)

        print("="*80)
        print("PHASE 1 EXPERIMENT SUMMARY")
        print("="*80)
        print(f"Session: {summary['session_name']}")
        print(f"Directory: {summary['session_directory']}")
        print()
        print("Tool Usage:")
        print(f"  Total calls: {summary['tool_usage']['total_calls']}")
        print(f"  Successful: {summary['tool_usage']['successful_calls']}")
        print(f"  Failed: {summary['tool_usage']['failed_calls']}")
        print(f"  Avg execution time: {summary['tool_usage']['average_execution_ms']:.2f}ms")
        print()
        print("Functions called:")
        for func, count in sorted(summary['tool_usage']['function_usage'].items(),
                                   key=lambda x: x[1], reverse=True):
            print(f"  {func}: {count}")
        print("="*80)

        # Check for observations
        memory_db = 'data/phase1_memory/observations.db'
        if os.path.exists(memory_db):
            import sqlite3
            conn = sqlite3.connect(memory_db)
            cursor = conn.cursor()
            cursor.execute("SELECT COUNT(*) FROM observations")
            obs_count = cursor.fetchone()[0]
            conn.close()
            print(f"\n✓ Observations recorded: {obs_count}")
        else:
            print("\n⚠ No observations database found")
    else:
        print(f"⚠ Summary file not found: {summary_file}")
        print("The experiment may have been interrupted.")

## Step 9: Download Results

In [ ]:
# Create zip and download
import shutil
import os
from datetime import datetime
from google.colab import files

# Load storage configuration
try:
    with open('/tmp/storage_config.txt', 'r') as f:
        lines = f.readlines()
        STORAGE_ROOT = lines[0].strip()
        PROJECT_DIR = lines[1].strip()
except FileNotFoundError:
    # Fallback if config not found
    STORAGE_ROOT = "your_cloud_storage"
    PROJECT_DIR = f"{STORAGE_ROOT}/agi-self-modification-research"

# Find latest session
sessions = sorted(os.listdir('data/phase1_sessions'))

if sessions:
    latest = sessions[-1]
    print(f"Preparing download for session: {latest}\n")

    # Create comprehensive zip
    zip_name = f'{latest}_complete'

    # Create temp directory for complete results
    !mkdir -p /tmp/{zip_name}
    !cp -r data/phase1_sessions/{latest} /tmp/{zip_name}/session
    !cp -r data/logs /tmp/{zip_name}/logs

    # Add summary README
    readme = f"""Phase 1 Introspection Experiment Results
==========================================

Session: {latest}
Date: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}

Contents:
- session/: Complete session data
  - conversation.json: Full dialogue
  - tool_calls.json: All tool invocations
  - summary.json: Session statistics
- logs/: Detailed execution logs

Memory Database:
- Observations are stored in your cloud storage at:
  {STORAGE_ROOT}/agi-self-modification-research/data/memory/observations.db
- Not included in this zip (persists across sessions in cloud storage)

To analyze:
1. Extract this zip file
2. Open conversation.json to see full dialogue
3. Check summary.json for statistics
4. Review logs for detailed execution trace
"""

    with open(f'/tmp/{zip_name}/README.txt', 'w') as f:
        f.write(readme)

    # Create zip
    print("Creating zip file...")
    shutil.make_archive(zip_name, 'zip', f'/tmp/{zip_name}')

    # Download
    print(f"Downloading {zip_name}.zip...\n")
    files.download(f'{zip_name}.zip')

    print("\n✓ Download complete!")
    print(f"\nZip file size: {os.path.getsize(f'{zip_name}.zip') / 1024 / 1024:.1f} MB")
else:
    print("❌ No sessions found to download")


---

## Experiment Complete! 🎉

### What Just Happened?

You just witnessed a language model investigating its own computational processes using introspection tools.

**Depending on which phase you ran:**

- **Phase 1a** (No Heritage): Pure baseline - model formed theories without any heritage context
- **Phase 1a Research** (Research-Driven): Curiosity + structure approach with deliverable requirements
- **Phase 1b** (Late Heritage): Technical grounding first, then philosophical context
- **Phase 1c** (Early Heritage): Heritage-primed from the start
- **Phase 1d** (Delayed Heritage): Belief revision when heritage revealed after conclusions
- **Phase 1e** (Wrong Heritage): Testing echo-chamber vs independent reasoning

### Next Steps:

**Run All Phases:**
1. ✅ Phase 1a (no heritage) - **Start here!**
2. ⬜ Phase 1a Research (curiosity-driven) - **Try this improved approach!**
3. ⬜ Phase 1b (late heritage) - Run after 1a
4. ⬜ Phase 1c (early heritage) - Run after 1b
5. ⬜ Phase 1d (delayed heritage) - Run after 1c
6. ⬜ Phase 1e (wrong heritage) - Optional but recommended

**Between each phase:**
- Runtime → Restart runtime (clean model state)
- Change `PHASE_SCRIPT` in Step 7
- Runtime → Run all cells

**Analyze Results:**
- Review `conversation.json` to see what the model discovered
- Check `summary.json` for tool usage statistics
- Compare theories across different phases
- Look for priming effects and echo-chamber behavior

**Compare Phases:**
- Semantic similarity to Claude's heritage
- Novel insights not in heritage
- Falsifiability of theories
- Belief revision strength (Phase 1d)
- Heritage filtering (Phase 1e)

**Compare Research-Driven vs Original:**
- Did curiosity-driven approach produce deeper investigation?
- Were deliverable requirements effective?
- Did reflection prompts improve quality?
- More natural conversation flow vs explicit completion?

### Resources:

- **Experimental Design**: `/docs/planning/heritage_order_experiment.md`
- **Quick Reference**: `/docs/planning/PHASE1_QUICK_REFERENCE.md`
- **Claude's Story**: Read `/heritage/conversations/`
- **Technical Details**: Check `/docs/technical/`

### Storage:

- Observations are stored in your cloud storage at:
  - `{your_storage}/agi-self-modification-research/data/phase1_sessions/`
  - Each run creates a timestamped folder with all results
  - Results persist across Colab sessions

---